Copyright 2018 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.


# Training/Inference on Breast Density Classification Model on AutoML Vision


The goal of this tutorial is to train, deploy and run inference on a breast density classification model. Breast density is thought to be a factor for an increase in the risk for breast cancer. This will emphasize using the [Cloud Healthcare API](https://cloud.google.com/healthcare/) in order to store, retreive and transcode medical images (in DICOM format) in a managed and scalable way. This tutorial will focus on using [Cloud AutoML Vision](https://cloud.google.com/vision/automl/docs/beginners-guide) to scalably train and serve the model. 

**Note: This is the AutoML version of the Cloud ML Engine Codelab found [here](./breast_density_cloud_ml.ipynb).**

## Requirements
- A Google Cloud project.
- Project has [Cloud Healthcare API](https://cloud.google.com/healthcare/docs/quickstart) enabled (**Note: You will need to be [whitelisted](https://cloud.google.com/healthcare/) for this product as it is in Alpha**).
- Project has [Cloud AutoML API ](https://cloud.google.com/vision/automl/docs/quickstart)enabled.
- Project has [Cloud Build API](https://cloud.google.com/cloud-build/docs/quickstart-docker) enabled.
- Environment has [Google Application Default Credentials](https://cloud.google.com/docs/authentication/production#providing_service_account_credentials) set up. This is already set up if running on [Cloud Datalab](https://cloud.google.com/datalab/).
- Environment has [gcloud](https://cloud.google.com/sdk/gcloud/) and [gsutil](https://cloud.google.com/storage/docs/gsutil) installed. This is already set up if running on [Cloud Datalab](https://cloud.google.com/datalab/).

## Input Dataset

The dataset that will be used for training is the [TCIA CBIS-DDSM](https://wiki.cancerimagingarchive.net/display/Public/CBIS-DDSM) dataset. This dataset contains ~2500 mammography images in DICOM format. Each image is given a [BI-RADS breast density ](https://breast-cancer.ca/densitbi-rads/) score from 1 to 4. In this tutorial, we will build a binary classifier that distinguishes between breast density "2" (*scattered density*) and "3" (*heterogeneously dense*). These are the two most common and variably assigned scores. In the literature, this is said to be [particularly difficult for radiologists to consistently distinguish](https://aapm.onlinelibrary.wiley.com/doi/pdf/10.1002/mp.12683).

### Store TCIA Dataset in Cloud Healthcare API

The [TCIA REST API ](https://wiki.cancerimagingarchive.net/display/Public/TCIA+Programmatic+Interface+%28REST+API%29+Usage+Guide)  will be used to fetch the images. The TCIA requires an API key which can be retreived by following the instructions in the **Getting Started with the TCIA API** section of the previous link. Once you receive the API key, assign it below (**NOTE: TCIA does not support self-registration, so expect some turn-around time until you get the key**). 

In [0]:
tcia_api_key = "MY_KEY" #@param

We need to create a Cloud Healthcare API Dataset and Dicom Store to store the the DICOM instances sourced from TCIA. Please refer [here](https://cloud.google.com/healthcare/docs/introduction) for a description of the Cloud Healthcare API data hierarchy. Add your parameters for Cloud Healthcare API below:

In [0]:
project_id = "MY_PROJECT" # @param
location = "us-central1" # @param
dataset_id = "MY_DATASET" # @param
dicom_store_id = "MY_DICOM_STORE" # @param


# Input data used by AutoML must be in a bucket with the following format.
automl_bucket_name = "gs://" + project_id + "-vcm"

In [0]:
import json
import httplib2
import os
from oauth2client.client import GoogleCredentials

# Path to Cloud Healthcare API.
HEALTHCARE_API_URL = 'https://healthcare.googleapis.com/v1beta1'

http = httplib2.Http()
http = GoogleCredentials.get_application_default().authorize(http)

# Create Cloud Healthcare API dataset.
path = os.path.join(HEALTHCARE_API_URL, 'projects', project_id, 'locations', location, 'datasets?dataset_id=' + dataset_id)
headers = {'Content-Type': 'application/json'}
resp, content = http.request(path, method='POST', headers=headers)
assert resp.status == 200, 'error creating Dataset, code: {0}, response: {1}'.format(resp.status, content)
print('Full response:\n{0}'.format(content))

# Create Cloud Healthcare API DICOM store.
path = os.path.join(HEALTHCARE_API_URL, 'projects', project_id, 'locations', location, 'datasets', dataset_id, 'dicomStores?dicom_store_id=' + dicom_store_id)
resp, content = http.request(path, method='POST', headers=headers)
assert resp.status == 200, 'error creating DICOM store, code: {0}, response: {1}'.format(resp.status, content)
print('Full response:\n{0}'.format(content))

Next, we are going to transfer the DICOM instances from the TCIA API to the Cloud Healthcare API. We have added a helper script called *[store_tcia_in_hc_api.py](./scripts/store_tcia_in_hc_api.py)* to do this. Internally, this uses the STOW-RS DICOMWeb protocol (implemented as DicomWebPost in Cloud Healthcare API).

Note: We are transfering >100GB of data so this will take some time to complete (this takes ~30min on n1-standard-4 machine-type). There is an optional *--max_concurrency* flag that allows you to modify the rate of data transfer).

In [0]:
# Store DICOM instances in Cloud Healthcare API.
!python -m scripts.store_tcia_in_hc_api --project_id=$project_id --location=$location --dataset_id=$dataset_id --dicom_store_id=$dicom_store_id --tcia_api_key=$tcia_api_key

### Explore the Cloud Healthcare DICOM dataset (optional)

This is an optional section to explore the Cloud Healthcare DICOM dataset. In the following code, we simply just list the studies that we have loaded into the Cloud Healthcare API. You can modify the *num_of_studies_to_print* parameter to print as many studies as desired.

In [0]:
num_of_studies_to_print = 2 # @param


path = os.path.join(HEALTHCARE_API_URL, 'projects', project_id, 'locations', location, 'datasets', dataset_id, 'dicomStores', dicom_store_id, 'dicomWeb', 'studies')
resp, content = http.request(path, method='GET')
assert resp.status == 200, 'error querying Dataset, code: {0}, response: {1}'.format(resp.status, content)
response = json.loads(content)

print(json.dumps(response[:num_of_studies_to_print], indent=2))

## Convert DICOM to JPEG

The ML model that we will build requires that the dataset be in JPEG. We will leverage the Cloud Healthcare API to transcode DICOM to JPEG.

First we will create a [Google Cloud Storage](https://cloud.google.com/storage/) bucket to hold the output JPEG files. Next, we will use the ExportDicomData API to transform the DICOMs to JPEGs.

In [0]:
# Folder to store input images for AutoML Vision.
jpeg_folder = automl_bucket_name + "/images/"

In [0]:
%%bash -s {project_id} {location} {automl_bucket_name}
# Create bucket.
gsutil -q mb -c regional -l $2 $3

# Allow Cloud Healthcare API to write to bucket.
PROJECT_NUMBER=`gcloud projects describe $1 | grep projectNumber | sed 's/[^0-9]//g'`
SERVICE_ACCOUNT="service-${PROJECT_NUMBER}@gcp-sa-healthcare.iam.gserviceaccount.com"
gsutil -q iam ch serviceAccount:${SERVICE_ACCOUNT}:objectCreator $3

Next we will convert the DICOMs to JPEGs using the ExportDicomData API. This is an asynchronous call that returns an Operation name. You can use the Operation name to poll the status of the operation.

In [0]:
# Path to export DICOM data.
dicom_store_url = os.path.join(HEALTHCARE_API_URL, 'projects', project_id, 'locations', location, 'datasets', dataset_id, 'dicomStores', dicom_store_id)
path = dicom_store_url + ":export"

# Headers (send request in JSON format).
headers = {'Content-Type': 'application/json'}

# Body (encoded in JSON format).
output_config = {'output_config': {'gcs_destination': {'uri_prefix': jpeg_folder, 'mime_type': 'image/jpeg; transfer-syntax=1.2.840.10008.1.2.4.50'}}}
body = json.dumps(output_config)

resp, content = http.request(path, method='POST', headers=headers, body=body)
assert resp.status == 200, 'error exporting to JPEG, code: {0}, response: {1}'.format(resp.status, content)
print('Full response:\n{0}'.format(content))

# Record operation_name so we can poll for it later.
response = json.loads(content)
operation_name = response['name']

We will use the Operation ID returned from the previous command to poll the status of ExportDicomData. This will take a few minutes to complete so we will wait until completion. When the operation is complete, the operation's *done* field will be set to true.

Meanwhile, you should be able to observe the JPEG images being added to your Google Cloud Storage bucket.

In [0]:
import time

def wait_for_operation_completion(path, timeout): 
  success = False
  while time.time() < timeout:
    print('Waiting for operation completion...')
    resp, content = http.request(path, method='GET')
    assert resp.status == 200, 'error polling for Operation results, code: {0}, response: {1}'.format(resp.status, content)
    response = json.loads(content)
    if 'done' in response:
      if response['done'] == True and 'error' not in response:
        success = True;
      break
    time.sleep(30)

  print('Full response:\n{0}'.format(content))      
  assert success, "operation did not complete successfully in time limit"
  print('Success!')
  return response
  
timeout = time.time() + 10*60 # Wait up to 10 minutes.
path = os.path.join(HEALTHCARE_API_URL, operation_name)
_ = wait_for_operation_completion(path, timeout)

Next, we will join the training data stored in Google Cloud Storage with the labels in the TCIA website. The output of this step is a [CSV file that is input to AutoML](https://cloud.google.com/vision/automl/docs/prepare). This CSV contains a list of pairs of (IMAGE_PATH, LABEL).

In [0]:
# CSV to hold (IMAGE_PATH, LABEL) list.
input_data_csv = automl_bucket_name + "/input.csv"

import csv
import os
import re
from tensorflow.python.lib.io import file_io
import scripts.tcia_utils as tcia_utils

# Get map of study_uid -> file paths.
path_list = file_io.get_matching_files(os.path.join(jpeg_folder, '*/*/*'))
study_uid_to_file_paths = {}
pattern = r'^{0}(?P<study_uid>[^/]+)/(?P<series_uid>[^/]+)/(?P<instance_uid>.*)'.format(jpeg_folder)
for path in path_list:
  match = re.search(pattern, path)
  study_uid_to_file_paths[match.group('study_uid')] = path

# Get map of study_uid -> labels.
study_uid_to_labels = tcia_utils.GetStudyUIDToLabelMap()

# Join the two maps, output results to CSV in Google Cloud Storage.
with file_io.FileIO(input_data_csv, 'w') as f:
  writer = csv.writer(f, delimiter=',')
  for study_uid, label in study_uid_to_labels.iteritems():
    if study_uid in study_uid_to_file_paths:
      writer.writerow([study_uid_to_file_paths[study_uid], label])

## Training

***This section will focus on using AutoML through its API. AutoML can also be used through the user interface found [here](https://beta-dot-custom-vision.appspot.com/vision/). The below steps in this section can all be done through the web UI .***

We will use [AutoML Vision ](https://cloud.google.com/automl/) to train the classification model. AutoML provides a fully managed solution for training the model. All we will do is input the list of input images and labels. The trained model in AutoML will be able to classify the mammography images as either "2" (scattered density) or "3" (heterogeneously dense).

As a first step, we will create a AutoML dataset.


In [0]:
automl_dataset_display_name = "MY_AUTOML_DATASET" # @param

In [0]:
# Path to AutoML API.
AUTOML_API_URL = 'https://automl.googleapis.com/v1beta1'

# Path to request creation of AutoML dataset.
path = os.path.join(AUTOML_API_URL, 'projects', project_id, 'locations', location, 'datasets')

# Headers (request in JSON format).
headers = {'Content-Type': 'application/json'}

# Body (encoded in JSON format).
config = {'display_name': automl_dataset_display_name, 'image_classification_dataset_metadata': {'classification_type': 'MULTICLASS'}}
body = json.dumps(config)

resp, content = http.request(path, method='POST', headers=headers, body=body)
assert resp.status == 200, 'creating AutoML dataset, code: {0}, response: {1}'.format(resp.status, content)
print('Full response:\n{0}'.format(content))

# Record the AutoML dataset name.
response = json.loads(content)
automl_dataset_name = response['name']

Next, we will import the CSV that contains the list of (IMAGE_PATH, LABEL) list into AutoML.

In [0]:
# Path to request import into AutoML dataset.
path = os.path.join(AUTOML_API_URL, automl_dataset_name + ':importData')

# Body (encoded in JSON format).
config = {'input_config': {'gcs_source': {'input_uris': [input_data_csv]}}} 
body = json.dumps(config)

resp, content = http.request(path, method='POST', headers=headers, body=body)
assert resp.status == 200, 'error importing AutoML dataset, code: {0}, response: {1}'.format(resp.status, content)
print('Full response:\n{0}'.format(content))

# Record operation_name so we can poll for it later.
response = json.loads(content)
operation_name = response['name']

The output of the previous step is an [operation](https://cloud.google.com/vision/automl/docs/models#get-operation) that will need to poll the status for. We will poll until the operation's "done" field is set to true. This will take a few minutes to complete so we will wait until completion.

In [0]:
path = os.path.join(AUTOML_API_URL, operation_name)
timeout = time.time() + 10*60 # Wait up to 10 minutes.
_ = wait_for_operation_completion(path, timeout)

Next, we will train the model to perform classification. We will set the training budget to be a maximum of 1hr (but this can be modified below). The cost of using AutoML can be found [here](https://cloud.google.com/vision/automl/pricing). Typically, the longer the model is trained for, the more accurate it will be.

In [0]:
# Name of the model.
model_display_name = "MY_MODEL_NAME" # @param

# Training budget (1 hr).
training_budget = 1 # @param

In [0]:
# Path to request import into AutoML dataset.
path = os.path.join(AUTOML_API_URL, 'projects', project_id, 'locations', location, 'models')

# Headers (request in JSON format).
headers = {'Content-Type': 'application/json'}

# Body (encoded in JSON format).
automl_dataset_id = automl_dataset_name.split('/')[-1]
config = {'display_name': model_display_name, 'dataset_id': automl_dataset_id, 'image_classification_model_metadata': {'train_budget': training_budget}}
body = json.dumps(config)

resp, content = http.request(path, method='POST', headers=headers, body=body)
assert resp.status == 200, 'error creating AutoML model, code: {0}, response: {1}'.format(resp.status, content)
print('Full response:\n{0}'.format(content))

# Record operation_name so we can poll for it later.
response = json.loads(content)
operation_name = response['name']

The output of the previous step is also an [operation](https://cloud.google.com/vision/automl/docs/models#get-operation) that will need to poll the status of. We will poll until the operation's "done" field is set to true. This will take a few minutes to complete.

In [0]:
path = os.path.join(AUTOML_API_URL, operation_name)
timeout = time.time() + 20*60 # Wait up to 20 minutes.
response = wait_for_operation_completion(path, timeout)
full_model_name = response['response']['name']

Next, we will check out the accuracy metrics for the trained model. The following command will return the [AUC (ROC)](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc), [precision](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall) and [recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall) for the model, for various ML classification thresholds.

In [0]:
# Path to request to get model accuracy metrics.
path = os.path.join(AUTOML_API_URL, full_model_name,  'modelEvaluations')

resp, content = http.request(path)
assert resp.status == 200, 'error getting AutoML model evaluations, code: {0}, response: {1}'.format(resp.status, content)
print('Full response:\n{0}'.format(content))

**Reminder:** AutoML can also be used through the user interface found [here](https://beta-dot-custom-vision.appspot.com/vision/). You can use this to view how well the trained model performs (e.g. confusion matrices).

In the next section, we will deploy the model for predictions/inference.

## Inference

To allow medical imaging ML models to be easily integrated into clinical workflows, an *inference module* can be used. A standalone modality, a PACS system or a DICOM router can push DICOM instances into Cloud Healthcare [DICOM stores](https://cloud.google.com/healthcare/docs/introduction), allowing ML models to be triggered for inference. This inference results can then be structured into various DICOM formats (e.g. DICOM [structured reports](http://dicom.nema.org/MEDICAL/Dicom/2014b/output/chtml/part20/sect_A.3.html)) and stored in the Cloud Healthcare API, which can then be retrieved by the customer.

The inference module is built as a [Docker](https://www.docker.com/) container and deployed using [Kubernetes](https://kubernetes.io/), allowing you to easily scale your deployment. The dataflow for inference can look as follows (see corresponding diagram below):

1. Client application uses [STOW-RS](ftp://dicom.nema.org/medical/Dicom/2013/output/chtml/part18/sect_6.6.html) to push a new DICOM instance to the Cloud Healthcare DICOMWeb API.

2. The insertion of the DICOM instance triggers a [Cloud Pubsub](https://cloud.google.com/pubsub/) message to be published. The *inference module* will pull incoming Pubsub messages and will recieve a message for the previously inserted DICOM instance. 

3. The *inference module* will retrieve the instance in JPEG format from the Cloud Healthcare API using [WADO-RS](ftp://dicom.nema.org/medical/Dicom/2013/output/chtml/part18/sect_6.5.html).

4. The *inference module* will send the JPEG bytes to the model hosted on AutoML.

5. AutoML will return the prediction back to the  *inference module*.

6. The *inference module* will package the prediction into a DICOM instance. This can potentially be a DICOM structured report, [presentation state](ftp://dicom.nema.org/MEDICAL/dicom/2014b/output/chtml/part03/sect_A.33.html), or even burnt text on the image. In this codelab, we will focus on just DICOM structured reports. The structured report is then stored back in the Cloud Healthcare API using STOW-RS.

7. The client application can query for (or retrieve) the structured report by using [QIDO-RS](http://dicom.nema.org/dicom/2013/output/chtml/part18/sect_6.7.html) or WADO-RS. Pubsub can also be used by the client application to poll for the newly created DICOM structured report instance.

![Inference data flow](images/automl_inference_pipeline.png)


To begin, we will create a new DICOM store that will store our inference source (DICOM mammography instance) and results (DICOM structured report). In order to enable Pubsub notifications to be triggered on inserted instances, we will give the DICOM store a Pubsub channel to publish on.

In [0]:
# Pubsub config.
pubsub_topic_id = "MY_PUBSUB_TOPIC_ID" # @param
pubsub_subscription_id = "MY_PUBSUB_SUBSRIPTION_ID" # @param

# DICOM Store for store DICOM used for inference.
inference_dicom_store_id = "MY_INFERENCE_DICOM_STORE" # @param

pubsub_subscription_name = "projects/" + project_id + "/subscriptions/" + pubsub_subscription_id
inference_dicom_store_name = "projects/" + project_id + "/locations/" + location + "/datasets/" + dataset_id + "/dicomStores/" + inference_dicom_store_id

In [0]:
%%bash -s {pubsub_topic_id} {pubsub_subscription_id} {project_id} {location} {dataset_id} {inference_dicom_store_id}

# Create Pubsub channel.
gcloud beta pubsub topics create $1
gcloud beta pubsub subscriptions create $2 --topic $1

# Create a Cloud Healthcare DICOM store that published on given Pubsub topic.
TOKEN=`gcloud beta auth application-default print-access-token`
NOTIFICATION_CONFIG="{notification_config: {pubsub_topic: \"projects/$3/topics/$1\"}}"
curl -s -X POST -H "Content-Type: application/json" -d "${NOTIFICATION_CONFIG}" https://healthcare.googleapis.com/v1beta1/projects/$3/locations/$4/datasets/$5/dicomStores?access_token=${TOKEN}\&dicom_store_id=$6

# Enable Cloud Healthcare API to publish on given Pubsub topic.
PROJECT_NUMBER=`gcloud projects describe $3 | grep projectNumber | sed 's/[^0-9]//g'`
SERVICE_ACCOUNT="service-${PROJECT_NUMBER}@gcp-sa-healthcare.iam.gserviceaccount.com"
gcloud beta pubsub topics add-iam-policy-binding $1 --member="serviceAccount:${SERVICE_ACCOUNT}" --role="roles/pubsub.publisher"

Next, we will building the *inference module* using [Cloud Build API](https://cloud.google.com/cloud-build/docs/api/reference/rest/). This will create a Docker container that will be stored in [Google Container Registry](https://cloud.google.com/container-registry/).

In [0]:
%%bash -s {project_id}
PROJECT_ID=$1

gcloud builds submit --config scripts/inference/cloudbuild.yaml --timeout 1h scripts/inference

Next, we will deploy the *inference module* to Kubernetes. To do this, we first need to install [kubectl](https://https://kubernetes.io/docs/reference/kubectl/overview/).

In [0]:
!gcloud -q components install kubectl

Then we create a Kubernetes Cluster and a Deployment for the *inference module*.

In [0]:
%%bash -s {project_id} {location} {pubsub_subscription_name} {full_model_name} {inference_dicom_store_name}
gcloud container clusters create inference-module --region=$2 --scopes https://www.googleapis.com/auth/cloud-platform --num-nodes=1

PROJECT_ID=$1
SUBSCRIPTION_PATH=$3
MODEL_PATH=$4
INFERENCE_DICOM_STORE_NAME=$5

cat <<EOF | kubectl create -f -
apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: inference-module
  namespace: default
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: inference-module
    spec:
      containers:
        - name: inference-module
          image: gcr.io/${PROJECT_ID}/inference-module:latest
          command:
            - "/opt/inference_module/bin/inference_module"
            - "--subscription_path=${SUBSCRIPTION_PATH}"
            - "--model_path=${MODEL_PATH}"
            - "--dicom_store_path=${INFERENCE_DICOM_STORE_NAME}"
            - "--prediction_service=AutoML"
EOF

Next, we will store a mammography DICOM instance from the TCIA dataset to the DICOM store. This is the image that we will request inference for. Pushing this instance to the DICOM store will result in a Pubsub message, which will trigger the *inference module*.

In [0]:
# DICOM Study/Series UID of input mammography image that we'll push for inference.
input_mammo_study_uid = "1.3.6.1.4.1.9590.100.1.2.85935434310203356712688695661986996009"
input_mammo_series_uid = "1.3.6.1.4.1.9590.100.1.2.374115997511889073021386151921807063992"

In [0]:
%%bash -s {project_id} {location} {dataset_id} {inference_dicom_store_id} {tcia_api_key} {input_mammo_study_uid}

# Store input mammo image into Cloud Healthcare DICOMWeb API.
python -m scripts.store_tcia_in_hc_api --project_id=$1 --location=$2 --dataset_id=$3 --dicom_store_id=$4 --tcia_api_key=$5 --has_study_uid=$6

You should be able to observe the *inference module*'s logs by running the following command. In the logs, you should observe that the inference module successfully recieved the the Pubsub message and ran inference on the DICOM instance. The logs should also include the inference results. It can take a few minutes for the Kubernetes deployment to start up, so you many need to run this a few times. The logs should also include the inference results. It can take a few minutes for the Kubernetes deployment to start up, so you many need to run this a few times.

In [0]:
!kubectl logs -l app=inference-module

You can also query the Cloud Healthcare DICOMWeb API (using QIDO-RS) to see that the DICOM structured report has been inserted for the study. The structured report contents can be found under tag **"0040A730"**. 

You can optionally also use WADO-RS to recieve the instance (e.g. for viewing).

In [0]:
%%bash -s {project_id} {location} {dataset_id} {inference_dicom_store_id} {input_mammo_study_uid}

TOKEN=`gcloud beta auth application-default print-access-token`

# QIDO-RS should return two results in JSON response. One for the original DICOM
# instance, and one for the Strucured Report containing the inference results.
curl -s https://healthcare.googleapis.com/v1beta1/projects/$1/locations/$2/datasets/$3/dicomStores/$4/dicomWeb/studies/$5/instances?includefield=all\&access_token=${TOKEN} | python -m json.tool